In [ ]:
import nltk
nltk.download("brown")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

#1

In [ ]:
import gensim
from nltk.corpus import brown
import random

def read_corpus(data, tokens_only=False):
    for i, doc_id in enumerate(data):
        words = brown.words(fileids=doc_id)
        tokens = gensim.utils.simple_preprocess(' '.join(words))
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

categories = brown.categories()

train_corpus = [fileids for category in categories for fileids in brown.fileids(categories=category)]
random.shuffle(train_corpus)
test_corpus = train_corpus[:len(train_corpus)//10]
train_corpus = train_corpus[len(train_corpus)//10:]

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
train_corpus_tagged = list(read_corpus(train_corpus))
model.build_vocab(train_corpus_tagged)
model.train(train_corpus_tagged, total_examples=model.corpus_count, epochs=model.epochs)

def classify_documents(model, test_corpus):
    for doc_id in test_corpus:
        words = brown.words(fileids=doc_id)
        test_tokens = gensim.utils.simple_preprocess(' '.join(words))
        inferred_vector = model.infer_vector(test_tokens)
        similar_docs = model.docvecs.most_similar([inferred_vector], topn=3)
        print(f"Document: {doc_id}, Category: {brown.categories(fileids=doc_id)[0]}")
        print(f"Most similar documents:")
        for sim_doc_id, similarity in similar_docs:
            print(f"- Document ID: {train_corpus[sim_doc_id]}, Similarity: {similarity}")

classify_documents(model, test_corpus)


<ipython-input-8-34022479a70f>:31: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_docs = model.docvecs.most_similar([inferred_vector], topn=3)


Document: cb21, Category: editorial
Most similar documents:
- Document ID: cd11, Similarity: 0.67829829454422
- Document ID: cb20, Similarity: 0.6379902958869934
- Document ID: cb02, Similarity: 0.6181785464286804
Document: cb11, Category: editorial
Most similar documents:
- Document ID: ca37, Similarity: 0.7319857478141785
- Document ID: cb20, Similarity: 0.6810892820358276
- Document ID: cb25, Similarity: 0.6796932816505432
Document: cp24, Category: romance
Most similar documents:
- Document ID: ca13, Similarity: 0.7695900797843933
- Document ID: ca15, Similarity: 0.7195014357566833
- Document ID: ca39, Similarity: 0.7142265439033508
Document: cf15, Category: lore
Most similar documents:
- Document ID: cd10, Similarity: 0.7316956520080566
- Document ID: cj23, Similarity: 0.68105149269104
- Document ID: cd02, Similarity: 0.6486451625823975
Document: ce28, Category: hobbies
Most similar documents:
- Document ID: ch27, Similarity: 0.7422364354133606
- Document ID: ce30, Similarity: 0.72

#2

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from nltk.corpus import names, movie_reviews
import nltk
import numpy as np
import pandas as pd

In [2]:
twenty_users = datasets.fetch_20newsgroups()
df = pd.DataFrame({'doc': twenty_users.data, 'target': twenty_users.target})

In [3]:
vectorizer = TfidfVectorizer(stop_words='english',smooth_idf=True)
input_matrix = vectorizer.fit_transform(df['doc'])

In [17]:
cosine_similarity_without_stopwords=cosine_similarity(input_matrix)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(input_matrix, df['target'], test_size=0.2, random_state=42)

logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, y_train)


LogisticRegression()

In [6]:
train_accuracy = logreg_classifier.score(X_train, y_train)
test_accuracy = logreg_classifier.score(X_test, y_test)

In [7]:
print(train_accuracy)
print(test_accuracy)

0.98342724560822
0.8983650022094565


Model with using the stop words(from Homework 4)

In [8]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
traindf, testdf = train_test_split(df)

from nltk.tokenize import word_tokenize
import nltk

distribution = nltk.FreqDist(word for doc in traindf['doc'] for word in word_tokenize(doc))
total_words = len(traindf['doc'])
words975 = [word for word, freq in distribution.items() if freq / total_words >= 0.975 or freq / total_words <= 0.025]

vectorizer = TfidfVectorizer(stop_words=words975)
X_train = vectorizer.fit_transform(traindf['doc'])
X_test = vectorizer.transform(testdf['doc'])

cosine_similarity_with_stopwords = cosine_similarity(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['00000315', '000007', '000021', '000042', '000062david42', '000094', '0000vec', '00014', '000152', '0001x7c', '000246', '000256', '000359', '00041032', '000413', '00044513', '0005', '0005111312na1em', '0005111312na3em', '000531', '000601', '0007', '000851', '0009', '00090711', '000956', '000ds', '000mi', '000miles', '000rpm', '000s', '000th', '001', '0010', '00100111b', '001004', '0010580b', '001116', '001125', '001127', '001211', '001230', '001319', '001321', '001338', '001428', '001555', '001642', '001707', '001718', '001757', '001813', '001934', '0020', '002118', '0022', '002222', '0023', '002302', '0024', '002651', '0028', '002811', '0029', '002937', '002d', '0030', '003015', '003029', '003221', '003258u19250', '0033', '0034', '003522', '003719', '003749', '0038', '003800', '003848', '00

In [14]:
print(cosine_similarity_with_stopwords )

[[1.         0.03671606 0.03869442 ... 0.02145829 0.06959022 0.02379447]
 [0.03671606 1.         0.0265034  ... 0.02289733 0.05875822 0.03936711]
 [0.03869442 0.0265034  1.         ... 0.02486235 0.04440788 0.02894483]
 ...
 [0.02145829 0.02289733 0.02486235 ... 1.         0.03297273 0.28691912]
 [0.06959022 0.05875822 0.04440788 ... 0.03297273 1.         0.04360725]
 [0.02379447 0.03936711 0.02894483 ... 0.28691912 0.04360725 1.        ]]


In [15]:
classifier = LogisticRegression()
classifier.fit(X_train, traindf['target'])

train_accuracy = accuracy_score(traindf['target'], classifier.predict(X_train))
print(train_accuracy)

test_accuracy = accuracy_score(testdf['target'], classifier.predict(X_test))
print(test_accuracy)

0.943429581614614
0.8115942028985508


Comparison between 6 documents

In [19]:
indices = [0, 10, 20, 30, 40, 50]
with_stopwords = cosine_similarity_with_stopwords[indices][:, indices]
without_stopwords = cosine_similarity_without_stopwords[indices][:, indices]

print(without_stopwords)

[[1.         0.00965885 0.02092331 0.09779586 0.00634752 0.00628715]
 [0.00965885 1.         0.00218438 0.03178546 0.00130573 0.00282087]
 [0.02092331 0.00218438 1.         0.00894163 0.01390446 0.00760715]
 [0.09779586 0.03178546 0.00894163 1.         0.00930175 0.02564995]
 [0.00634752 0.00130573 0.01390446 0.00930175 1.         0.01377305]
 [0.00628715 0.00282087 0.00760715 0.02564995 0.01377305 1.        ]]


In [20]:
print(with_stopwords)

[[1.         0.01548588 0.08451565 0.07150948 0.02418779 0.00667087]
 [0.01548588 1.         0.01011199 0.04009233 0.04378713 0.02252989]
 [0.08451565 0.01011199 1.         0.05771457 0.05520981 0.0352386 ]
 [0.07150948 0.04009233 0.05771457 1.         0.10239255 0.04157255]
 [0.02418779 0.04378713 0.05520981 0.10239255 1.         0.00679181]
 [0.00667087 0.02252989 0.0352386  0.04157255 0.00679181 1.        ]]


The test accuracy with the removal of stop words comes as 0.89, which is higher than the testing accuracy that came up as 0.82 when the stop words were added.

The word model distance can be seen from the cosine_similarity matrix from both cases.

The values tend to be lower in case of stop words removal, as the stop words occur in documents, so the removal of them essentially improves the accuracy by focusing more on the meaningful content of the documents.

With the removal of stop words the similarity values are less meaning that the similarity between the documents are far less than the documents that have the stop words.

This is also the reason for higher accuracy with the stop words removal as the similarity is less and the prediction is more accurate.